In [1]:
import cv2
import numpy as np
import os
import pickle
from datetime import datetime

# Directories
dataset_dir = "dataset"
model_dir = "models"
os.makedirs(dataset_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

# Haar cascade for detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [2]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.2, 5)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)

    cv2.imshow('Face Detection (press q to quit)', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [3]:
label = "Patience"   # change to your name
num_images = 50
save_path = os.path.join(dataset_dir, label)
os.makedirs(save_path, exist_ok=True)

cap = cv2.VideoCapture(0)
count = 0

while count < num_images:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.2, 5)

    for (x, y, w, h) in faces:
        roi = gray[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
        filename = os.path.join(save_path, f"{datetime.now().strftime('%Y%m%d_%H%M%S')}.png")
        cv2.imwrite(filename, roi)
        count += 1
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)

    cv2.imshow('Collecting Faces', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print(f"Saved {count} images to {save_path}")

Saved 50 images to dataset\Patience


In [9]:
images, labels = [], []
label_ids = {}
current_id = 0

for name in os.listdir(dataset_dir):
    person_path = os.path.join(dataset_dir, name)
    if not os.path.isdir(person_path):
        continue
    if name not in label_ids:
        label_ids[name] = current_id
        current_id += 1
    for img_name in os.listdir(person_path):
        path = os.path.join(person_path, img_name)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            images.append(img)
            labels.append(label_ids[name])

images = np.array(images)
labels = np.array(labels)

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.train(images, labels)
recognizer.save(os.path.join(model_dir, 'face_model.xml'))

with open(os.path.join(model_dir, 'labels.pkl'), 'wb') as f:
    pickle.dump(label_ids, f)

print(f"Trained on {len(images)} images. Labels: {label_ids}")


AttributeError: module 'cv2' has no attribute 'face'

In [ ]:
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read(os.path.join(model_dir, 'face_model.xml'))

with open(os.path.join(model_dir, 'labels.pkl'), 'rb') as f:
    label_ids = pickle.load(f)
id_to_name = {v: k for k, v in label_ids.items()}

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.2, 5)

    for (x, y, w, h) in faces:
        roi = gray[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
        id_, conf = recognizer.predict(roi)
        name = id_to_name.get(id_, "Unknown")
        cv2.putText(frame, f"{name} ({conf:.0f})", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)

    cv2.imshow('Face Recognition', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()